<a href="https://colab.research.google.com/github/SonnyDev/llm-apps-langchain/blob/main/LangChain_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nous devrions considérer le LLM comme un moteur de raisonnement, dans lequel nous pouvons lui fournir des morceaux de texte ou d'autres sources d'information, et ensuite le LLM utilisera peut-être ces connaissances de fond qu'il a apprises sur Internet, mais pour utiliser les nouvelles informations que nous lui donnons pour nous aider à répondre à des questions, à réfléchir sur le contenu ou même à décider de la marche à suivre. Et c'est ce que le cadre des Agents de LangChain nous aide à faire.

Les agents sont l'une des parties les plus puissantes de LangChain. Ils font également partie des éléments les plus récents, donc beaucoup de choses émergent : ce que sont les agents, comment les créer et les utiliser, comment les équiper avec différents types d'outils comme les moteurs de recherche intégrés à LangChain, et également comment créer nos propres outils, afin que nous puissions permettre aux agents d'interagir avec n'importe quelles bases de données, n'importe quelles API, n'importe quelles fonctions que nous pourrions vouloir qu'ils utilisent.

In [ ]:
#from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
#from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Nous voulons utiliser le LLM comme un moteur de raisonnement de l'agent, où il se connecte à d'autres sources telles que des données et des calculs. Nous souhaitons que ce moteur de raisonnement soit aussi bon et précis que possible. C'est pourquoi nous réglons la température à 0.

In [ ]:
llm = ChatOpenAI(temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

Ensuite, nous chargeons quelques outils :

- L'outil llm-math, qui est une chaîne utilisant un LLM en conjonction avec une calculatrice pour résoudre des problèmes mathématiques.
- L'outil Wikipedia, qui est une API se connectant à Wikipedia, nous permettant d'effectuer des recherches sur Wikipedia et d'obtenir des résultats.

In [ ]:
!pip install numexpr #llm-math nécessite ce package
!pip install wikipedia


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
tools = load_tools(['llm-math', 'wikipedia'], llm = llm)

Ensuite, nous initialisons l'agent.

Description de CHAT_ZERO_SHOT_REACT :
- CHAT : un agent optimisé pour les modèles de chat.
- REACT : une technique de prompting conçue pour obtenir la meilleure performance de raisonnement possible de la part du LLM.
- Gestion des erreurs de parsing : si la sortie n'est pas dans le format souhaité, nous renvoyons la sortie au LLM et lui demandons de se corriger.

In [ ]:
agent = initialize_agent(
        tools,
        llm,
        agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        handle_parsing_errors = True,
        verbose = True)

In [ ]:
agent("Combien vaut 10+10")



> Entering new AgentExecutor chain...
Je vais utiliser la calculatrice pour répondre à cette question.

Thought: La question demande la somme de 10 et 10.
Action:
```
{
  "action": "Calculator",
  "action_input": "10+10"
}
```

Observation: Answer: 20
Thought:La réponse est 20.
Final Answer: 20

> Finished chain.


{'input': 'Combien vaut 10+10', 'output': '20'}

In [ ]:
agent("Quelle pays a remporté la coupe du monde de Rugby 2023")



> Entering new AgentExecutor chain...
Thought: I need to find out which country won the Rugby World Cup in 2023.
Action:
```json
{
  "action": "Wikipedia",
  "action_input": "Rugby World Cup 2023"
}
```
Observation: Page: 2023 Rugby World Cup
Summary: The 2023 Rugby World Cup (French: Coupe du monde de rugby 2023) was the  tenth men's Rugby World Cup, the quadrennial world championship for national rugby union teams. It took place in France from 8 September to 28 October 2023 in nine venues across the country. The opening game and final took place at the Stade de France, north of Paris. The tournament was held in the bicentenary year of the purported invention of the sport by William Webb Ellis.The tournament was scheduled to last six weeks, but in February 2021 World Rugby added a week to provide additional rest days for player welfare. This meant that teams had a minimum of five days' rest for all matches. It was the fourth time France has hosted the Rugby World Cup, having previou

{'input': 'Quelle pays a remporté la coupe du monde de Rugby 2023',
 'output': 'South Africa'}

### Toolkits : Pandas DataFrame

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [ ]:
import pandas as pd
from langchain.llms import OpenAI

df = pd.read_csv("titanic.csv")

In [ ]:
# Utiliser ZERO_SHOT_REACT_DESCRIPTION
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True) # Nécessite le package tabulate

In [ ]:
# Utiliser un agent de type OpenAIFunctions
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [ ]:
agent.run("Combien il y a-il de lignes ?")



> Entering new AgentExecutor chain...
Thought: Je dois trouver le nombre de lignes dans le dataframe.
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 891
Thought: Je sais maintenant le nombre de lignes.
Final Answer: Il y a 891 lignes.

> Finished chain.


'Il y a 891 lignes.'

In [ ]:
agent.run("Combien d'hommes et de femmes il y a-il ?")



> Entering new AgentExecutor chain...
Thought: Je dois compter le nombre de lignes pour chaque sexe
Action: python_repl_ast
Action Input: df['Sex'].value_counts()
Observation: Sex
male      577
female    314
Name: count, dtype: int64
Thought: Je connais maintenant le nombre d'hommes et de femmes
Final Answer: Il y a 577 hommes et 314 femmes.

> Finished chain.


'Il y a 577 hommes et 314 femmes.'

In [ ]:
agent.run("Combien de femmes ont survécu ?")



> Entering new AgentExecutor chain...
Thought: Je dois trouver le nombre de femmes qui ont survécu.
Action: python_repl_ast
Action Input: df[(df['Sex'] == 'female') & (df['Survived'] == 1)].shape[0]
Observation: 233
Thought: Je connais maintenant le nombre de femmes qui ont survécu.
Final Answer: 233 femmes ont survécu.

> Finished chain.


'233 femmes ont survécu.'